In [1]:
import base64
import requests,json

COCO_INSTANCE_CATEGORY_train2017 = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# img 2 base64
with open("test_image5.jpg", "rb") as image_file:
    base64str = base64.b64encode(image_file.read()).decode("utf-8")

# send img to api
payload = json.dumps({
  "base64str": base64str,
})

headers = {
  'Content-Type': 'application/json'
}

response = requests.put("http://172.16.120.15/proxy/u9e39a57/predict/",data = payload, headers=headers)
data_dict = response.json()

In [2]:
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

def PILImage_to_cv2(img):
    return np.asarray(img)

def drawboundingbox(img, boxes,pred_cls, rect_th=2, text_size=1, text_th=2):
    img = PILImage_to_cv2(img)
    class_color_dict = {}
    
    #initialize some random colors for each class for better looking bounding boxes
    for cat in pred_cls:
        class_color_dict[cat] = [random.randint(0, 255) for _ in range(3)]
    
    for i in range(len(boxes)):
        cv2.rectangle(img, (int(boxes[i][0]), int(boxes[i][1])),
                      (int(boxes[i][2]),int(boxes[i][3])),
                      color=class_color_dict[pred_cls[i]], thickness=rect_th)
        cv2.putText(img,str(COCO_INSTANCE_CATEGORY_train2017[pred_cls[i]]), (int(boxes[i][0]), int(boxes[i][1])),  cv2.FONT_HERSHEY_SIMPLEX, text_size, class_color_dict[pred_cls[i]],thickness=text_th) # Write the prediction class
    plt.figure(figsize=(20,30))
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()
    
img = Image.open("test_image5.jpg")
drawboundingbox(img, data_dict['boxes'], data_dict['classes'])